In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_staging.stg_users
USING DELTA
AS
SELECT
  time,
  users.my_submit.id AS submit_id,
  users.my_submit.datetime AS submit_datetime,
  users.my_review.id AS review_id,
  users.my_review.datetime AS review_datetime,
  users.my_approval.id AS approval_id,
  users.my_approval.datetime AS approval_datetime,
  users.my_status.status AS status
FROM workspace.silver_schema.silver_delta_table;

MERGE INTO workspace.gold_staging.stg_users AS target
USING (
  SELECT * FROM (
    SELECT
      time,
      users.my_submit.id AS submit_id,
      users.my_submit.datetime AS submit_datetime,
      users.my_review.id AS review_id,
      users.my_review.datetime AS review_datetime,
      users.my_approval.id AS approval_id,
      users.my_approval.datetime AS approval_datetime,
      users.my_status.status AS status,
      ROW_NUMBER() OVER (
        PARTITION BY time, users.my_submit.id
        ORDER BY time DESC
      ) AS rn
    FROM workspace.silver_schema.silver_delta_table
  ) WHERE rn = 1
) AS source
ON target.submit_id = source.submit_id
AND target.time = source.time
WHEN MATCHED THEN
  UPDATE SET
    target.submit_datetime = source.submit_datetime,
    target.review_id = source.review_id,
    target.review_datetime = source.review_datetime,
    target.approval_id = source.approval_id,
    target.approval_datetime = source.approval_datetime,
    target.status = source.status
WHEN NOT MATCHED THEN
  INSERT (
    time,
    submit_id,
    submit_datetime,
    review_id,
    review_datetime,
    approval_id,
    approval_datetime,
    status
  )
  VALUES (
    source.time,
    source.submit_id,
    source.submit_datetime,
    source.review_id,
    source.review_datetime,
    source.approval_id,
    source.approval_datetime,
    source.status
  );
